In [2]:
%run ../LibrariesAndFunctions/LibrariesFunctions.ipynb

C:\Users\tasos\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tasos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tasos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
partOfBooksForLdaUp60DescTitle = pd.read_csv('../Data/descPrepOnly.csv')

### Column "Name" cleaning

In [4]:
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["Name"].apply(removeNewlines)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(removeHtmlTags)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(removeLinks)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(lowerCasing)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(reducingIncorrectCharacterRepeatation)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(expandContractions)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(removeSpecialCharacters)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(removeNumbers)
partOfBooksForLdaUp60DescTitle["NameProcessed"] = partOfBooksForLdaUp60DescTitle["NameProcessed"].apply(removeWhitespace)

partOfBooksForLdaUp60DescTitle["RemovedAuthorNameFromNames"] = [' '.join(x for x in a.split() if x not in set(b.split())) for a, b in zip(partOfBooksForLdaUp60DescTitle['NameProcessed'], partOfBooksForLdaUp60DescTitle['Authors'])]

C:\Users\antonia\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


### Tokenization column "NameProcessed"

In [5]:
partOfBooksForLdaUp60DescTitle["TokenizeName"] = partOfBooksForLdaUp60DescTitle["RemovedAuthorNameFromNames"].apply(sent2Words)

### Lemmatization column "NameProcessed"

In [6]:
partOfBooksForLdaUp60DescTitle["TokenLemName"] = partOfBooksForLdaUp60DescTitle["TokenizeName"].apply(lemmatizationAll, flag = 1)

partOfBooksForLdaUp60DescTitle['TokenLem'] = partOfBooksForLdaUp60DescTitle['TokenLem'].apply(strToListOfStrs)
partOfBooksForLdaUp60DescTitle["TokenLemAll"] = partOfBooksForLdaUp60DescTitle['TokenLem'] + partOfBooksForLdaUp60DescTitle["TokenLemName"]

In [7]:
partOfBooksForLdaUp60DescTitle.to_csv("partOfBooksForLdaUp60DescTitle.csv", index = False)

In [8]:
def replacenWithAnd(text):
    return([word if word != 'n' else 'and' for word in text])

In [9]:
partOfBooksForLdaUp60DescTitle["TokenLem"] = partOfBooksForLdaUp60DescTitle["TokenLem"].apply(replacenWithAnd)

### Create bigrmas and trigrams

In [12]:
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

bigramFinder = nltk.collocations.BigramCollocationFinder.from_documents([des for des in partOfBooksForLdaUp60DescTitle["TokenLemAll"]])
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_documents([des for des in partOfBooksForLdaUp60DescTitle["TokenLemAll"]])

In [13]:
# Filter for bigrams with only noun-type structures
def bigramFilter(bigram):
    tag = nltk.pos_tag(bigram)
    if(not(tag[0][1].startswith('J') or tag[0][1].startswith('N')) and not tag[1][1].startswith('N')):
        return False
    if bigram[0] in stopWords or bigram[1] in stopWords:
        return False
    if (len(bigram[0])<=2 or len(bigram[1])<= 2) and bigram[1] != "fi":
        return False
    return True

# Filter for trigrams with only noun-type structures
def trigramFilter(trigram):
    tag = nltk.pos_tag(trigram)
    if(not (tag[0][1].startswith('J') or tag[0][1].startswith('N')) and not (tag[2][1].startswith('J') or tag[2][1].startswith('N'))):
        return False
    if trigram[0] in stopWords or trigram[2] in stopWords:
        return False
    if (len(trigram[0]) <= 2 and len(trigram[1]) <= 2) or (len(trigram[0]) <=2 and len(trigram[2]) <= 2) or (len(trigram[1]) <= 2 and len(trigram[2]) <= 2) or (trigram[1] == 's'):
        return False
    return True 

In [13]:
nltk.pos_tag(["play", "basketball", "again"])

[('play', 'NN'), ('basketball', 'NN'), ('again', 'RB')]

In [10]:
tag = nltk.pos_tag(["play", "basketball", "again"])

### NLTK BIGRAMS AND TRIGRAMS FREQUENCY

In [14]:
bigramFreq = bigramFinder.ngram_fd.items()

trigramFreq = trigramFinder.ngram_fd.items()

In [15]:
bigramFreqTable = pd.DataFrame(list(bigramFreq), columns=['Bigram','Freq']).sort_values(by='Freq', ascending=False).reset_index(drop=True)[:600]
trigramFreqTable = pd.DataFrame(list(trigramFreq), columns=['Trigram','Freq']).sort_values(by='Freq', ascending=False).reset_index(drop=True)[:600]

In [16]:
filteredBigramFreq = bigramFreqTable[bigramFreqTable.apply(lambda bigram: bigramFilter(bigram['Bigram']) and bigram.Freq > 50, axis = 1)]
filteredTrigramFreq = trigramFreqTable[trigramFreqTable.apply(lambda trigram: trigramFilter(trigram['Trigram']) and trigram.Freq > 50, axis = 1)]

In [17]:
bigramsFreq = [' '.join(x) for x in filteredBigramFreq.Bigram.values]
bigramsFreq[:10]

['united state',
 'world war',
 'year old',
 'twentieth century',
 'first time',
 'case study',
 'wide range',
 'civil war',
 'york time',
 'nineteenth century']

In [18]:
trigramsFreq = [' '.join(x) for x in filteredTrigramFreq.Trigram.values]
trigramsFreq[:10]

['step by step',
 'world war ii',
 'around the world',
 'men and woman',
 'need to know',
 'along the way',
 'work be culturally',
 'believe this work',
 'fall in love',
 'point of view']

### NLTK BIGRAMS AND TRIGRAMS PMI 


In [19]:
bigramFinder.apply_freq_filter(60)
bigramScoresPmi = bigramFinder.score_ngrams(bigrams.pmi)

trigramFinder.apply_freq_filter(60)
trigramScoresPmi = trigramFinder.score_ngrams(trigrams.pmi)

In [20]:
bigramPmi = pd.DataFrame(bigramScoresPmi)
bigramPmi.columns = ['Bigram', 'Pmi']
bigramPmi.sort_values(by='Pmi', axis = 0, ascending = False, inplace = True)

In [21]:
trigramPmi = pd.DataFrame(trigramScoresPmi)
trigramPmi.columns = ['Trigram', 'Pmi']
trigramPmi.sort_values(by='Pmi', axis = 0, ascending = False, inplace = True)

In [22]:
filteredBigramPmi = bigramPmi[bigramPmi.apply(lambda bigram: bigramFilter(bigram['Bigram']) and bigram.Pmi > np.percentile(bigramPmi.Pmi, 25), axis = 1)][:500]
filteredTrigramPmi = trigramPmi[trigramPmi.apply(lambda trigram: trigramFilter(trigram['Trigram']) and trigram.Pmi > np.percentile(bigramPmi.Pmi, 25), axis = 1)][:500]

In [23]:
bigramsPmi = [' '.join(x) for x in filteredBigramPmi.Bigram.values]
bigramsPmi[:10]

['dalai lama',
 'feng shui',
 'saddam hussein',
 'nova scotia',
 'sri lanka',
 'hercule poirot',
 'costa rica',
 'notre dame',
 'sci fi',
 'roller coaster']

In [24]:
trigramsPmi = [' '.join(x) for x in filteredTrigramPmi.Trigram.values]
trigramsPmi[:10]

['edgar allan poe',
 'arthur conan doyle',
 'frank lloyd wright',
 'partial differential equation',
 'mark notation marginalia',
 'unc press endure',
 'errant mark etc',
 'luther king jr',
 'martin luther king',
 'poor picture errant']

### NLTK BIGRAMS AND TRIGRAMS CHI SQUARE

In [25]:
bigramFinder.apply_freq_filter(60)
bigramScoresChiSquare = bigramFinder.score_ngrams(bigrams.chi_sq)

trigramFinder.apply_freq_filter(60)
trigramScoresChiSquare = trigramFinder.score_ngrams(trigrams.chi_sq)

In [26]:
bigramChi = pd.DataFrame(list(bigramScoresChiSquare))
bigramChi.columns = ['Bigram', 'Chi_sq']
bigramChi.sort_values(by='Chi_sq', axis = 0, ascending = False, inplace = True)

In [27]:
trigramChi = pd.DataFrame(list(trigramScoresChiSquare))
trigramChi.columns = ['Trigram', 'Chi_sq']
trigramChi.sort_values(by='Chi_sq', axis = 0, ascending = False, inplace = True)

In [28]:
filteredBigramChi = bigramChi[bigramChi.apply(lambda bigram: bigramFilter(bigram['Bigram']) and bigram.Chi_sq > np.percentile(bigramChi.Chi_sq, 25), axis = 1)][:500]
filteredTrigramChi = trigramChi[trigramChi.apply(lambda trigram: trigramFilter(trigram['Trigram']) and trigram.Chi_sq > np.percentile(trigramChi.Chi_sq, 25), axis = 1)][:500]

In [29]:
bigramsChi = [' '.join(x) for x in filteredBigramChi.Bigram.values]
bigramsChi[:10]

['avant garde',
 'los angeles',
 'feng shui',
 'hong kong',
 'scarce antiquarian',
 'dalai lama',
 'notation marginalia',
 'sri lanka',
 'hercule poirot',
 'roller coaster']

In [30]:
trigramsChi = [' '.join(x) for x in filteredTrigramChi.Trigram.values]
trigramsChi[:10]

['edgar allan poe',
 'mark notation marginalia',
 'arthur conan doyle',
 'frank lloyd wright',
 'partial differential equation',
 'errant mark etc',
 'unc press endure',
 'luther king jr',
 'martin luther king',
 'princeton legacy library']

### Μerging bigrams and trigrams, calculated from the three collocation algorithms and cleaning the duplicates

In [31]:
bigrams60DescTitle = list(set(bigramsPmi + bigramsChi + bigramsFreq))
bigrams60DescTitle

['homicide detective',
 'high quality',
 'los angeles',
 'wide variety',
 'martial art',
 'phone call',
 'pocket size',
 'dynamical system',
 'everyday life',
 'death penalty',
 'birthday party',
 'special attention',
 'color illustration',
 'ronald reagan',
 'economic growth',
 'color plate',
 'julius caesar',
 'data mining',
 'page turner',
 'civil liberty',
 'university press',
 'weight loss',
 'complete idiot',
 'hong kong',
 'specially commission',
 'prime suspect',
 'master storyteller',
 'care provider',
 'full length',
 'hotel room',
 'scotland yard',
 'harsh reality',
 'firsthand account',
 'stock market',
 'pay tribute',
 'virtual reality',
 'world war',
 'particle physic',
 'arrange alphabetically',
 'golden age',
 'molecular biology',
 'adam smith',
 'million dollar',
 'lavishly illustrate',
 'bottom line',
 'public opinion',
 'non profit',
 'comic strip',
 'building block',
 'private investigator',
 'fifty year',
 'pharmaceutical industry',
 'finite element',
 'rhode islan

In [32]:
trigrams60DescTitle = list(set(trigramsPmi + trigramsChi + trigramsFreq))
trigrams60DescTitle

['race and gender',
 'theory and practice',
 'provide a concise',
 'want to learn',
 'everything you need',
 'twenty first century',
 'curated for quality',
 'two year old',
 'trace the origin',
 'never before see',
 'late eighteenth century',
 'major motion picture',
 'late nineteenth century',
 'set the stage',
 'declaration of independence',
 'last ten year',
 'economic and political',
 'collection of essay',
 'throughout the text',
 'marginalia and flaw',
 'york time bestselling',
 'physical and emotional',
 'literature in affordable',
 'past two decade',
 'professor of english',
 'variety of setting',
 'across the globe',
 'cover such topic',
 'provide a unique',
 'principle and practice',
 'easy to find',
 'princeton legacy library',
 'year old son',
 'central and eastern',
 'social and cultural',
 'mid th century',
 'life on earth',
 'nut and bolt',
 'trace the history',
 'tool you need',
 'stop at nothing',
 'back into print',
 'unlock the secret',
 'trace the evolution',
 'awa

In [33]:
def replaceNgram(text):
    for gram in bigrams60DescTitle:
        text = text.replace(gram, '_'.join(gram.split()))
    for gram in trigrams60DescTitle:
        text = text.replace(gram, '_'.join(gram.split()))
    return text

In [34]:
partOfBooksForLdaUp60DescTitle["TokenLemBigram"] = partOfBooksForLdaUp60DescTitle.TokenLemAll.apply(listOfStingsToString)

In [36]:
partOfBooksForLdaUp60DescTitle["TokenLemBigramTrigram"] = partOfBooksForLdaUp60DescTitle["TokenLemBigram"].map(lambda text: replaceNgram(text))

### Join the words in the text that belong to the bigrams and trigrams lists 

In [37]:
englishNames = pd.read_csv('../Data/englishNames.txt', header=None)
englishNames = pd.DataFrame(englishNames.values, columns = ["Names"])
englishNames["Names"] = englishNames["Names"].apply(lowerCasing)
englishNames.shape

(18247, 1)

### Τokenize text , remove stop words, remove english names and remove words that have length less than or equal to two

In [38]:
partOfBooksForLdaUp60DescTitle["TokenLemBigramTrigramRemStopWords"] = partOfBooksForLdaUp60DescTitle.TokenLemBigramTrigram.map(lambda x: [word for word in x.split()
                                                    if (word not in stopWords)
                                                    and (len(word) > 2) and (word not in englishNames["Names"].values)])#na prosthesw remove most/less freq words

In [44]:
#Function that keep only the nouns from the already lemmatized data
def lemmatizationNouns(text):
    posDesc = nltk.pos_tag(text)
    lemmaWords = [word[0] for word in posDesc if (word[1].startswith('N')) and (word[1] not in bigrams60DescTitle) and (word[1] not in trigrams60DescTitle)]
    return lemmaWords


##################################################################################################
# Function that keep only the nouns and verbs from the already lemmatized data
def lemmatizationNounsAndVerbs(text):
    posDesc = nltk.pos_tag(text)
    lemmaWords = [word[0] for word in posDesc if (word[1].startswith('N') or word[1].startswith('V')) and (word[1] not in bigrams60DescTitle) and (word[1] not in trigrams60DescTitle)]
    return lemmaWords

### Keep nouns only

In [45]:
partOfBooksForLdaUp60DescTitle["TokenLemNounsBigramTrigramRemStopWords"] = partOfBooksForLdaUp60DescTitle.TokenLemBigramTrigramRemStopWords.map(lemmatizationNouns)

### Keep nouns and Verbs only

In [47]:
partOfBooksForLdaUp60DescTitle["TokenLemNounsVerbsBigramTrigramRemStopWords"] = partOfBooksForLdaUp60DescTitle.TokenLemBigramTrigramRemStopWords.map(lemmatizationNounsAndVerbs)

In [48]:
partOfBooksForLdaUp60DescTitle.to_csv('../Data/bookData.csv', index = False)

In [49]:
pd.DataFrame(bigrams60DescTitle, columns=['Bigrams']).to_csv("../Data/bigrams60DescTitle.csv", index=False)

In [50]:
pd.DataFrame(trigrams60DescTitle, columns=['Trigrams']).to_csv("../Data/trigrams60DescTitle.csv", index=False)

In [2]:
partOfBooksForLdaUp60DescTitle = pd.read_csv('../Data/bookData.csv')